# Scrape Fantasy Labs
This scrapes www.fantasylabs.com <br>
This website contains information on contest results that can be used to replicate DraftKings' salary and result files <br>

In [1]:
import requests
import datetime
import pandas as pd
from openpyxl import load_workbook
import os
import time
import numpy as np

import warnings
warnings.simplefilter(action="ignore")

baseball_path = r"C:\Users\james\Documents\MLB\Data"

In [2]:
# This reads in a map of team name, codes, and the number Fangraphs uses in their URLs
team_map = pd.read_csv(os.path.join(baseball_path, "Utilities", "Team Map.csv"))
# We just need teams right now
team_map = team_map[['FULLNAME', 'FANPROSTEAM', 'BBREFTEAM', 'ROTOWIRETEAM']]

In [32]:
# This scrapes the Fantasy Labs website to recreate DK salaries and results
def scrape_fantasylabs(dateStr):
    # Create date string
    dateStr_alpha = datetime.datetime.strptime(dateStr, '%Y-%M-%d').strftime('%Y%M%d')

    # Use date to find URL
    url = f'https://service.fantasylabs.com/contest-sources/?sport_id=3&date={dateStr}'
    # Use url to request json data
    jsonData = requests.get(url).json()
    # Gather groupId variable
    groupId = jsonData['contest-sources'][0]['draft_groups'][0]['id']
    print(jsonData)
   
    # Use that in this URL
    url2 = f'https://service.fantasylabs.com/live-contests/?sport=MLB&contest_group_id={groupId}'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}
    jsonData = requests.get(url2, headers=headers).json()
    
    # Extract contestIds
    contestIds = {}
    for each in jsonData['live_contests']:
        contestIds[each['contest_name']] = each['contest_id']
        # print(each['entry_cost'])
        
    # Gather standings
    # rows = []
    
    standings_list = []
    contestUsers_list = []
    fieldExposures_list = []
    for contestName, contestId in contestIds.items():
        url3 = f'https://dh5nxc6yx3kwy.cloudfront.net/contests/mlb/{dateStr_alpha}/{contestId}/lineups/'
        jsonData = requests.get(url3, headers=headers).json()

        rows = []
        lineups = jsonData['lineups']
        for k, v in lineups.items():
            v.update({'contestName':contestName})
            rows.append(v)

        standings = pd.DataFrame(rows)
    

        url4 = f'https://dh5nxc6yx3kwy.cloudfront.net/contests/mlb/{dateStr_alpha}/{contestId}/data/'
        jsonData = requests.get(url4).json()


        contestUsers = pd.DataFrame(jsonData['users']).T.reset_index(drop=True)
        

        fieldExposures = pd.DataFrame(jsonData['players']).T
        
        standings_list.append(standings)
        contestUsers_list.append(contestUsers)
        fieldExposures_list.append(fieldExposures)
        
    return standings_list, contestUsers_list, fieldExposures_list

In [191]:
standings_list, contestUsers_list, fieldExposures_list = scrape_fantasylabs("2023-04-09")

{'contest-sources': [{'id': 4, 'source_name': 'draftkings.com', 'is_primary': False, 'orphan_count': 0, 'ui_display_order': 1, 'display_name': 'DraftKings', 'min_salary': 0, 'max_salary': 0, 'is_active': True, 'short_name': 'dk', 'draft_groups': [{'id': 90972, 'contest_start_date': '2023-04-09T13:05:00', 'contest_end_date': '0001-01-01T00:00:00', 'contest_suffix': '', 'draft_group_id': 85265, 'game_count': 10, 'sport': {'id': 3}, 'source': {'id': 4}}]}, {'id': 8, 'source_name': 'draftkings.com showdown', 'is_primary': False, 'orphan_count': 0, 'ui_display_order': 2, 'display_name': 'DK Showdown (CPT)', 'min_salary': 0, 'max_salary': 0, 'is_active': True, 'short_name': '', 'draft_groups': [{'id': 90980, 'contest_start_date': '2023-04-09T19:08:00', 'contest_end_date': '0001-01-01T00:00:00', 'contest_suffix': '(SD vs ATL)', 'draft_group_id': 85275, 'game_count': 1, 'sport': {'id': 3}, 'source': {'id': 8}}]}]}


In [192]:
standings = standings_list[0]
standings = standings[standings['entryNameList'].astype('str').str.contains("gilesj12")]
    
standings[['points', 'totalSalary', 'payout', 'lineupRank', 'lineupHash', 'lineupPlayers', 'correlatedPlayers', 'teamStacks', 'lineupTrends']]

# for col in standings.columns:
#     print(col)

,points,totalSalary,payout,lineupRank,lineupHash,lineupPlayers,correlatedPlayers,teamStacks,lineupTrends
9033,27.90,16100,0.0,13585,91584:87830:159227,"{'C1': 91584, 'OF1': 87830, 'P1': 159227}",2,"{'4180': ['91584:0', '87830:0']}","{'qbPairedWithPassCatcher': False, 'qbStackPai..."
10405,29.71,33900,0.0,13360,75563:113107:6101:31666:181497:7541:31644,"{'1B1': 75563, '2B1': 113107, '3B1': 6101, 'C1...",5,"{'4166': ['75563:0', '113107:0', '6101:0', '31...","{'qbPairedWithPassCatcher': False, 'qbStackPai..."
10411,13.71,32000,0.0,14538,75563:113107:6101:91584:200000:7541:31603,"{'1B1': 75563, '2B1': 113107, '3B1': 6101, 'C1...",5,"{'4166': ['75563:0', '113107:0', '6101:0', '20...","{'qbPairedWithPassCatcher': False, 'qbStackPai..."
10427,27.71,20400,0.0,13620,91584:87830:181497:7541,"{'C1': 91584, 'OF1': 87830, 'OF2': 181497, 'P1...",2,"{'4180': ['91584:0', '87830:0']}","{'qbPairedWithPassCatcher': False, 'qbStackPai..."
12492,29.00,31200,0.0,13444,75563:113107:6101:91584:181497:158996:181526:3...,"{'1B1': 75563, '2B1': 113107, '3B1': 6101, 'C1...",7,"{'4166': ['75563:0', '113107:0', '6101:0', '18...","{'qbPairedWithPassCatcher': False, 'qbStackPai..."


In [190]:
contestUsers = contestUsers_list[0]
contestUsers.query('userId == "gilesj12"')['lineups'][2494]

{'87446:33839:112802:40007': {'lineupHash': '87446:33839:112802:40007',
  'lineupCt': 1},
 '7384:33015:255050:34563:112802:3734': {'lineupHash': '7384:33015:255050:34563:112802:3734',
  'lineupCt': 1},
 '7384:33015:255050:34563:33839:112802:3734': {'lineupHash': '7384:33015:255050:34563:33839:112802:3734',
  'lineupCt': 1},
 '33015:33839:112802:3734': {'lineupHash': '33015:33839:112802:3734',
  'lineupCt': 2}}

In [189]:
fieldExposures = fieldExposures_list[0].sort_values(by=['salary'], ascending=False)
fieldExposures.query('playerId == 7384')

,playerId,firstName,lastName,fullName,salary,position,rosterPosition,currentTeam,currentTeamId,eventId,eventTeamId,homeVisitor,favDog,projPoints,ownership,actualPoints,statDetails,madeCut
7384:0,7384,Justin,Turner,Justin Turner,3600,DH,1B,BOS,4162,5612410,56124102,Home,Dog,8.47,4.29,5.0,,0


In [66]:
standings_list[0].sort_values(by=['points'], ascending=False)
standings_list[0]
for i in standings_list[0]['entryNameList']:
    print(i)
# contestUsers_list[0].sort_values(by=['totalWinning'], ascending=False)

['chicksdigthewongball']
['PE21']
['wioux']
['chicksdigthewongball']
['lighty03s']
['Golf_Coach']
['kipperk25']
['sjw3']
['Latman7']
['brenarlo']
['lighty03s', '12PackAbs']
['sollydotcom']
['Tryhard2109']
['jsan04']
['bric75', 'giantsquid']
['giantsquid', 'yoshi0016']
['jbwilliams3']
['perihadion']
['hishboo']
['KevinRob2128']
['mikev305']
['shawnzhan']
['therealmccoy2984']
['needlunchmoney']
['inittobinkit']
['GuhReeDoe']
['youdacao']
['atrain00']
['Dshook420']
['wjgrimes']
['jonhoel']
['lethalbizzle']
['eiflerak']
['RankS69420']
['dakdakgoose']
['rwalkersc']
['kmcclain21', 'kristian24', 'maxwell517', 'bono6699', 'mayo843']
['solutionsports']
['wakeupmillionswife', 'Espn24hrs7']
['6foot2by245']
['cmiller8823']
['lzpnsx']
['KHICKE11']
['jacobstubbs', 'kmcclain21', 'neech74', 'jreilly', 'thedude2015', 'corypooler40']
['rd7242']
['roomtyme']
['Jjbamaboyjj']
['safetydance']
['GGallows', 'DreamchaserJC']
['ShaidyAdvice']
['tysonjsmith', 'csharrel1']
['spundin']
['J187']
['perihadion']
['sa

In [25]:
# This data is sloppy and some dates have to be fixed
def manual_clean(df):
    df = df[~((df['eventId'] == 5292910) & (df['currentTeam'] == 'SEA'))]
    df = df[~((df['eventId'] == 5292910) & (df['currentTeam'] == 'DET'))]
    # 8/19/2022
    df = df[~((df['eventId'] == 5374000) & (df['currentTeam'] == 'CHW'))]
    
    
    return df

In [26]:
# This creates basic Game Info to mimic DK Salaries away@home (ignoring date and time)
def create_gameinfo(df):
    df = df.groupby(['currentTeam', 'homeVisitor'])['eventId'].agg(pd.Series.mode).reset_index()
    
    df = df.merge(team_map, left_on='currentTeam', right_on='FANPROSTEAM', how='left')
    df['currentTeam'] = df['BBREFTEAM']
    
    df = manual_clean(df)
    
    # Create home and away dataframes
    home = df.query('homeVisitor == "Home"')
    visitor = df.query('homeVisitor == "Visitor"')
    # print(home[['currentTeam', 'eventId']])
    # print(visitor[['currentTeam', 'eventId']])
    # Merge them together
    merged = visitor.merge(home, on='eventId', how='inner')
    # Create Game Info variable
    merged['Game Info'] = merged['currentTeam_x'] + "@" + merged['currentTeam_y']
    merged.drop_duplicates(subset=['currentTeam_x'], inplace=True)
    merged.drop_duplicates(subset=['currentTeam_y'], inplace=True)
    
    # Just keep that and a game code to merge on
    merged = merged[['eventId', 'Game Info']]
    
    return merged

In [27]:
# This creates:

def create_dfs(standings, fieldExposures):
    # Split lineupHash to make each its own variable
    standings[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']] = standings['lineupHash'].str.split(":", expand=True)
    
    # Fill missings with a string
    standings.fillna("999909", inplace=True)
    
    # Gather player info
    # players = fieldExposures[['playerId', 'fullName', 'actualPoints', 'salary', 'position', 'rosterPosition', 'ownership']]
    players = fieldExposures.copy()
    players['playerId'] = players['playerId'].astype('str')
    players_cut = players[['playerId', 'position', 'fullName', 'actualPoints']]

    
    
    for i in range(1, 11):
        position = str(i)
        standings = standings.merge(players_cut, left_on=position, right_on='playerId', how='left', suffixes=("", position))

    standings.fillna("MI", inplace=True)
        
    standings['Lineup'] = ""
    for col in standings.columns:
        if col.startswith('position'):
            standings['Lineup'] = standings['Lineup'] + standings[col] + " " 
        if col.startswith('fullName'):
            standings['Lineup'] = standings['Lineup'] + standings[col] + " " 

    standings = standings.reset_index()

    # Duplicate lineups, one for each instance of that lineup
    standings = standings.loc[standings.index.repeat(standings.lineupCt)]
    
    standings['entryId'] = standings['index']
    standings['TimeRemaining'] = 0
    
    standings.rename(columns={'lineupRank':'Rank', 'entryId':'EntryId', 'entryNameList':'EntryName', 'points':'Points'}, inplace=True)
    
    results = standings[['Rank', 'EntryId', 'EntryName', 'TimeRemaining', 'Points', 'Lineup']]
    payout = standings[['Rank', 'EntryId', 'EntryName', 'TimeRemaining', 'Points', 'Lineup', 'payout', 'entry_cost']]
    
    
    
    players.rename(columns={'fullName':'Player', 'actualPoints':'FPTS', 'ownership':'%Drafted', 'rosterPosition': 'Roster Position', 'position':'Position', 'playerId':'ID'}, inplace=True)
    player_results = players[['Player', 'Roster Position', '%Drafted', 'FPTS']]
    players.rename(columns={'Player':'Name', 'FPTS':'AvgPointsPerGame', 'salary':'Salary'}, inplace=True)
    # print(players)
    players['Name + ID'] = players['Name'] + " (" + players['ID'].astype('str') + ")"
    
    # Create Game Info variable
    game_df = create_gameinfo(players)
    # Merge Game Info onto salaries
    merged = players.merge(game_df, on='eventId', how='left')
    merged.rename(columns={'currentTeam':'TeamAbbrev'}, inplace=True)
    # Keep relevant variables
    merged = merged[['Position', 'Name + ID', 'Name', 'ID', 'Roster Position', 'Salary', 'Game Info', 'TeamAbbrev', 'AvgPointsPerGame']]
    salaries = merged[['Position', 'Name + ID', 'Name', 'ID', 'Roster Position', 'Salary', 'Game Info', 'TeamAbbrev', 'AvgPointsPerGame']]
    
    player_results.sort_values(by=['FPTS'], ascending=False, inplace=True)
    salaries.sort_values(by=['Salary'], ascending=False, inplace=True)
    return results, payout, player_results, salaries


# results, payout, player_results, salaries = create_dfs(standings, fieldExposures)

In [33]:
# This creates the Salaries and Results csvs
def create_excels(dateStr):
    # Scrape fantasylabs for fieldExposures (which has salary info) and contestUsers (has lineups)
    fieldExposures, contestUsers, standings = scrape_fantasylabs(dateStr)
    
    # Create field lineup results and salaries
    results, payout, player_results, salaries = create_dfs(standings, fieldExposures)
        
    # Results (similar to DK results)
    date_short = dateStr.replace("-", "")
    filename = "Results " + date_short + ".xlsx"
    
    # Write as Excel 
    # We really want a CSV, but we have two DFs and I'm not sure how to write them correctly to a csv
    writer = pd.ExcelWriter(os.path.join(baseball_path, "Results Scraped2", filename), engine='openpyxl')
    
    results.to_excel(writer, sheet_name='Sheet1', header=True, index=False)
    player_results.to_excel(writer, sheet_name='Sheet1', header=True, index=False,
                 startcol=7,startrow=0)
    
    writer.save()
    
    # Convert it to CSV
    results = pd.read_excel(os.path.join(baseball_path, "Results Scraped2", filename))
    csvname = filename = "Results " + date_short + ".csv"
    results.to_csv(os.path.join(baseball_path, "Results Scraped2", csvname))
    
    # Salaries (similar to DK salaries)
    filename2 = "DKSalaries_" + date_short + ".csv"
    salaries.to_csv(os.path.join(baseball_path, "A7. Matchups - 1. Salaries", filename2))

    # Payouts
    filename3 = "Payouts_" + date_short + ".csv"
    payout.to_csv(os.path.join(baseball_path, "Payouts Scraped", filename3))
    
    

In [34]:
create_excels("2023-04-27")

{'contest-sources': [{'id': 4, 'source_name': 'draftkings.com', 'is_primary': False, 'orphan_count': 0, 'ui_display_order': 1, 'display_name': 'DraftKings', 'min_salary': 0, 'max_salary': 0, 'is_active': True, 'short_name': 'dk', 'draft_groups': [{'id': 92002, 'contest_start_date': '2023-04-27T18:40:00', 'contest_end_date': '0001-01-01T00:00:00', 'contest_suffix': '', 'draft_group_id': 86395, 'game_count': 5, 'sport': {'id': 3}, 'source': {'id': 4}}, {'id': 92000, 'contest_start_date': '2023-04-27T12:20:00', 'contest_end_date': '0001-01-01T00:00:00', 'contest_suffix': '(Early)', 'draft_group_id': 86387, 'game_count': 6, 'sport': {'id': 3}, 'source': {'id': 4}}]}]}
92002
2120


In [230]:
# Loop over dates that we have fangraphs projections for
for filename in os.listdir(r"C:\Users\james\Documents\MLB\Data\Rosters\Depth"):
    date = filename[5:13]
    print(date)
    
    dateStr = date[0:4] + "-" + date[4:6] + "-" + date[6:8]
    
    try:
        create_excels(dateStr)
    except:
        print("Didn't work")
    
    time.sleep(15)

20220401
Didn't work
20220402
Didn't work
20220403
Didn't work
20220404
Didn't work
20220405
Didn't work
20220406
Didn't work
20220407
20220408
20220409
20220410
20220411
20220412
20220413
20220414
20220415
20220416
20220417
20220418
20220419
20220420
20220421
20220422
20220423
20220424
20220425
20220426
20220427
20220428
20220429
20220430
20220501
20220502
20220503
20220504
20220505
20220506
20220507
20220508
20220509
20220510
20220511
20220512
20220513
20220514
20220515
20220516
20220517
20220518
20220519
20220520
20220521
20220522
20220523
20220524
20220525
20220526
20220527
Didn't work
20220528
Didn't work
20220529
20220530
20220531
20220601
20220602
20220603
20220604
20220605
20220606
20220607
20220608
20220609
20220610
20220611
20220612
20220613
20220614
20220615
20220616
20220617
20220618
20220619
20220620
20220621
20220622
20220623
20220624
Didn't work
20220625
20220626
20220627
20220628
20220629
20220630
20220701
20220702
20220703
20220704
20220705
20220706
20220707
20220708
2

In [419]:
# import time
# import bs4
# from bs4 import BeautifulSoup
# from selenium import webdriver

# chromedriver_path= r"C:\Users\james\Documents\MLB\chromedriver.exe"
# driver = webdriver.Chrome(chromedriver_path)
# url = "https://www.fantasylabs.com/mlb/contest-ownership/?date=04072022"
# driver.get(url)
# time.sleep(6) #if you want to wait 3 seconds for the page to load
# page_source = driver.page_source
# soup = bs4.BeautifulSoup(page_source, 'lxml')
# for tag in soup.find_all("div", id="ownershipGrid"):
#     print(tag.get_text(separator=" "))
    
# driver.close()
# # soup.prettify()

In [3]:
def player_data(slateID):
    slateID = str(slateID)
    url = 'https://www.rotowire.com/daily/tables/optimizer-mlb.php'
    payload = {
    'siteID': '1',
    'slateID': slateID,
    'projSource': 'RotoWire',
    'rst': 'RotoWire'
    }
    jsonData = requests.get(url, params=payload).json()
    df = pd.DataFrame(jsonData)
    
    df = df[['id', 'rotoPlayerID', 'player', 'ownership', 'proj_points', 'team', 'game_location']]
    
    
    return(df)

df = player_data(6721)
df
    

,id,rotoPlayerID,player,ownership,proj_points,team,game_location
0,12454,12454,Lucas Giolito,12.80,21.81,CWS,Away
1,15400,15400,Shane McClanahan,47.86,20.52,TB,Home
2,14312,14312,Zac Gallen,30.21,20.04,ARI,Away
3,14900,14900,Framber Valdez,19.37,17.82,HOU,Home
4,12492,12492,Jose Berrios,10.08,16.62,TOR,Away
...,...,...,...,...,...,...,...
206,9829,9829,Yasmani Grandal,1.12,0.00,CWS,Away
207,14293,14293,Brandon Marsh,1.18,0.00,PHI,Home
208,11460,11460,J.D. Martinez,0.00,0.00,LAD,Home
209,13740,13740,Franchy Cordero,0.00,0.00,BAL,Home


In [4]:
def matchup_data(slateID):
    url = f'https://www.rotowire.com/daily/tables/mlb/schedule.php?&siteID=1&slateID={slateID}'
    jsonData = requests.get(url).json()
    df = pd.DataFrame(jsonData)
    
    df = df[['date', 'visit_team', 'home_team']]

    
    return df

df = matchup_data(6721)
df

,date,visit_team,home_team
0,WED: 7:05 PM,CWS,BAL
1,WED: 7:05 PM,CIN,PHI
2,WED: 7:10 PM,TOR,BOS
3,WED: 7:10 PM,LAA,TB
4,WED: 8:05 PM,STL,CHC
5,WED: 8:10 PM,ARI,KC
6,WED: 8:10 PM,MIN,HOU
7,WED: 9:05 PM,MIL,LAD


In [5]:
def scrape_rotowire(slateID):
    player_df = player_data(slateID)
    matchup_df = matchup_data(slateID)
    
    df = player_df.merge(matchup_df, left_on='team', right_on=['visit_team'], how='left', suffixes=("", "away"))
    df = df.merge(matchup_df, left_on='team', right_on=['home_team'], how='left', suffixes=("", "home"))
    
    df['date'].fillna(df['datehome'],inplace=True)
    df['visit_team'].fillna(df['visit_teamhome'],inplace=True)
    df['home_team'].fillna(df['home_teamhome'],inplace=True)
    
    #Merge with team map
    df = df.merge(team_map, left_on='visit_team', right_on='ROTOWIRETEAM', how='left',suffixes=("", "away"))
    df = df.merge(team_map, left_on='home_team', right_on='ROTOWIRETEAM', how='left',suffixes=("", "home"))
    
    df['Game Info'] = df['BBREFTEAM'] + "@" + df['BBREFTEAMhome']
                          
    df['day'] = df['date'].str.slice(0,3)
    df['num_matchups'] = len(matchup_df)
        
    df = df[['id', 'player', 'Game Info', 'date', 'day', 'proj_points', 'ownership', 'num_matchups']]
    
    df['proj_points'] = pd.to_numeric(df['proj_points'])
    df['ownership'] = pd.to_numeric(df['ownership'])
    
    df['has_ownership'] = ((df['ownership'].mean()) > 0)
    df['has_ownership'] = df['has_ownership'].astype('int')
    
    df['slateID'] = slateID
        
    return df

df = scrape_rotowire(6721)
df

,id,player,Game Info,date,day,proj_points,ownership,num_matchups,has_ownership,slateID
0,12454,Lucas Giolito,CHW@BAL,WED: 7:05 PM,WED,21.81,12.80,8,1,6721
1,15400,Shane McClanahan,LAA@TBR,WED: 7:10 PM,WED,20.52,47.86,8,1,6721
2,14312,Zac Gallen,ARI@KCR,WED: 8:10 PM,WED,20.04,30.21,8,1,6721
3,14900,Framber Valdez,MIN@HOU,WED: 8:10 PM,WED,17.82,19.37,8,1,6721
4,12492,Jose Berrios,TOR@BOS,WED: 7:10 PM,WED,16.62,10.08,8,1,6721
...,...,...,...,...,...,...,...,...,...,...
206,9829,Yasmani Grandal,CHW@BAL,WED: 7:05 PM,WED,0.00,1.12,8,1,6721
207,14293,Brandon Marsh,CIN@PHI,WED: 7:05 PM,WED,0.00,1.18,8,1,6721
208,11460,J.D. Martinez,MIL@LAD,WED: 9:05 PM,WED,0.00,0.00,8,1,6721
209,13740,Franchy Cordero,CHW@BAL,WED: 7:05 PM,WED,0.00,0.00,8,1,6721


In [6]:
# 6721 is 8/24 (use pitcher, day)
# Not all will have ownership - maybe this is useful - might just project main slate (wrong)
# Run a bunch, put all together
# Read in player sims, add var for num_matchups, merge on player, num_matchups (once we know date)
# Stopped at 8300

In [ ]:
for i in range(6000, 2000, -1):
    print(i)
    filename = "slateID" + str(i) + ".csv"
    
    try:
        df = scrape_rotowire(i)
        df.to_csv(os.path.join(baseball_path, "RotoWire", filename))
        time.sleep(5)
    except:
        print("Doesn't work")
    
    
    
    

6000
5999
5998
5997
5996
5995
5994
5993
5992
5991
5990
5989
5988
5987
5986
5985
5984
5983
5982
5981
5980
5979
5978
5977
5976
5975
5974
5973
5972
5971
5970
5969
5968
5967
5966
5965
5964
5963
5962
5961
5960
5959
5958
5957
5956
5955
5954
5953
5952
5951
5950
5949
5948
5947
5946
5945
5944
5943
5942
5941
5940
5939
5938
5937
5936
5935
5934
5933
5932
5931
5930
5929
5928
5927
5926
5925
5924
5923
5922
5921
5920
5919
5918
5917
5916
5915
5914
5913
5912
5911
5910
5909
5908
5907
5906
5905
5904
5903
5902
5901
5900
5899
5898
5897
5896
5895
5894
5893
5892
5891
5890
5889
5888
5887
5886
5885
5884
5883
5882
5881
5880
5879
5878
5877
5876
5875
5874
5873
5872
5871
5870
5869
5868
5867
5866
5865
5864
5863
5862
5861
5860
5859
5858
5857
5856
5855
5854
5853
5852
5851
5850
5849
5848
5847
5846
5845
5844
5843
5842
5841
5840
5839
5838
5837
5836
5835
5834
5833
5832
5831
5830
5829
5828
5827
5826
5825
5824
5823
5822
5821
5820
5819
5818
5817
5816
5815
5814
5813
5812
5811
5810
5809
5808
5807
5806
5805
5804
5803
5802
5801


In [ ]:
# Create weekday dictionary to match with RotoWire's unhelpful date variable, which I converted to an also unhelpful day
weekday = {6:"SUN", 0:"MON", 1:"TUE", 2:"WED", 3:"THU", 4:"FRI", 5:"SAT"}

In [ ]:
# Read in player sims, try to figure out day
date = "20220407"
filename = "Player_Sims_" + date + ".csv"
sims = pd.read_csv(os.path.join(baseball_path, "Player Sims", filename))

team_list = (list(sims['Game Info'].unique()))
team_list.sort()

num_matchups = len(team_list)
print(datetime.datetime.strptime(date, '%Y%m%d').weekday())
dow = weekday[datetime.datetime.strptime(date, '%Y%m%d').weekday()]
num_matchups, dow


In [ ]:
def add_ownership(date):
    # Read in player sims, try to figure out day
    filename = "Player_Sims_" + date + ".csv"
    sims = pd.read_csv(os.path.join(baseball_path, "Player Sims", filename))

    # Game list could be used to merge, but idk
    team_list = (list(sims['Game Info'].unique()))
    team_list.sort()

    # Number of matchups
    num_matchups = len(team_list)
    # Day of week
    day = weekday[datetime.datetime.strptime(date, '%Y%m%d').weekday()]
    
    sims['num_matchups'] = num_matchups
    sims['day'] = day
    
    
    # Merge with giant list of ownership dataframes here

    

In [ ]:
def slate_guide():
    slate_stats_all = []
    for file in os.listdir(os.path.join(baseball_path, "RotoWire")):
        df = pd.read_csv(os.path.join(baseball_path, "RotoWire", file))
        
        slateID = df['slateID'][0]
        day = df['day'][0]
        has_ownership = df['has_ownership'][0]
        
        
        team_list = (list(df['Game Info'].unique()))
        team_list.sort()
        
        num_matchups = len(team_list)
        
        slate_stats = [slateID, day, has_ownership, num_matchups, team_list]
        slate_stats_all.append(slate_stats)
    slate_df = pd.DataFrame(slate_stats_all, columns=['slate', 'day', 'has_ownership', 'num_matchups', 'team_list'])
    
    slate_df['team_list'] = slate_df['team_list'].astype('str')
    
    return slate_df

slates = slate_guide()
slates

In [ ]:
def sims_guide():
    sims_stats_all = []
    for file in os.listdir(os.path.join(baseball_path, "Player Sims")):
        if file.endswith(".csv"):
            df = pd.read_csv(os.path.join(baseball_path, "Player Sims", file))

            team_list = (list(df['Game Info'].unique()))
            team_list.sort()
            
            date = file[12:20]

            num_matchups = len(team_list)
            day = weekday[datetime.datetime.strptime(date, '%Y%m%d').weekday()]
            
            sims_stats = [date, day, num_matchups, team_list]
            sims_stats_all.append(sims_stats)
    sims_df = pd.DataFrame(sims_stats_all, columns=['date', 'day', 'num_matchups', 'team_list'])
    
    sims_df['team_list'] = sims_df['team_list'].astype('str')
    
    return sims_df
    
sims_df = sims_guide()
sims_df            

In [ ]:
merged_df = slates.merge(sims_df, on=['day', 'num_matchups', 'team_list'], how='inner')
merged_df

In [ ]:
sims_df.query('date == "20220824"')

In [ ]:
slates.query('slate == 6721')